In [1]:
%load_ext autoreload
%autoreload 2

# Gaussian process regression for forecasting battery state of health

Automotive industry has undergone significant changes with innovations like electrification and Autonomous Driving (ADAS). Electrificaiton, in general talks about many aspects but electrifying powertrain is the main focus area. Electrification is supposed to reduce the carbon footprint of vehicles, or at least isolate them away from cities where the major population resides around the world. According to this report from [EV Volumes](https://www.ev-volumes.com/country/total-world-plug-in-vehicle-volumes/) database, the YoY growth of EV sales showed average monotinic increasing trends despite the COVID situation economically.

<img src="https://www.ev-volumes.com/wp-content/uploads/2021/01/WW-A-12-2020.png" width="800" />


The Li-ion battery (LiB) technology is the backbone of electrified powertrain. A strong and reliable LiB technology entails early and robust adaptation of electrified powertrain. To that end, LiB development has many challenges. One among these is the prediction of Age of LiB cells as it is a multiscale and multiphysics problem at core. If the LiB cell capacity history is known, forecasting the State of Health (SoH) is a challenge, because this is governed by many external and user specific aspects. 

In this [paper](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwjqmJ_855bvAhUVgOYKHR4lByAQFjAAegQIBBAD&url=https%3A%2F%2Fwww.sciencedirect.com%2Fscience%2Farticle%2Fpii%2FS0378775317306250&usg=AOvVaw1P31v8I4zlOJacXNwqw_xP), the authors have tried to make use of Gaussian Process Regression (GPR) for this forecasting task. This notebook attempts to recreate the results from the paper. 

This 

#Table of content

Table of content

> 1. Basic Single Output GP Results
>> 1.1. Kernel Function Selection
>>
>>1.2. Kernel Function Decomposition
>>
>>1.3. Short-term Lookahead Prediction
>>
>>1.4 Remaining Useful Life Prediction
> 2. Encoding exponential degradation via EMFs Results
> 3. Capturing Cell-to-Cell Correlations via Multi-output GPs Results

Some imports relevant to this project

In [2]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [3]:
from gpr.Kernels import Periodic

## Data Exploration 

# Results 

## Basic Single Output GP Results

### Kernel Function Selection

### Kernel Function Decomposition

### Short-term Lookahead Prediction

### Remaining Useful Life Prediction

## Encoding exponential degradation via EMFs Results

## Capturing Cell-to-Cell Correlations via Multi-output GPs Results

In [ ]:
def get_battery_data(name):
    data = pd.read_csv("./data/"+name+".csv", header=None)
    X_ = data[0].values.reshape(-1,1)
    y_ = data[1].values
    return X_, y_

In [ ]:
cell_names = ["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4"]
styles = ["--k","-k","-ok","-dk","-b","--b","-ob","-db"]
plt.figure(figsize=(20,10))
for i in range(len(cell_names)):
    X, y = get_battery_data(cell_names[i])
    plt.plot(X, y, styles[i], label=cell_names[i])
plt.legend()
plt.ylim(0.7,1)

In [ ]:
A1 = pd.read_csv("./data/B3.csv", header=None)
X_orig = A1[0].values.reshape(-1,1)
y_orig = A1[1].values

In [ ]:
learn_till = 100
X = X_orig[np.where(X_orig<learn_till)].reshape(-1,1)
y = y_orig[np.where(X_orig<learn_till)[0]]

In [ ]:
from sklearn.gaussian_process.kernels import ExpSineSquared, Matern, WhiteKernel, ExpSineSquared
from sklearn.gaussian_process import GaussianProcessRegressor

ma5 = Matern(length_scale=8e3,nu=5/2, length_scale_bounds=(1e-1, 5e3))
ma3 = Matern(length_scale=1e2,nu=3/2, length_scale_bounds=(1e-1, 5e2))
pe = ExpSineSquared(length_scale=1.0, periodicity=1.0, length_scale_bounds=(1e-05, 1e6), periodicity_bounds=(1e-05, 1e6))

kernel = ma5+ma3
gp = GaussianProcessRegressor(kernel=kernel,normalize_y=False, alpha=0.001, n_restarts_optimizer=20)

gp.fit(X, y)

In [ ]:
gp.log_marginal_likelihood_value_

In [ ]:
gp.kernel_.get_params

In [ ]:
x = np.atleast_2d(np.linspace(0,167,100)).T
y_pred, sigma = gp.predict(x, return_std=True)

In [ ]:
plt.figure(figsize=(6,5))
plt.plot(x, y_pred)
plt.fill(np.concatenate([x, x[::-1]]),np.concatenate([y_pred -  sigma,(y_pred + sigma)[::-1]]), alpha=0.1)
plt.plot(X_orig, y_orig, c='k')
plt.plot()
plt.ylim(0.5,1.05)
plt.axhline(y=0.775, c='k')
plt.axvline(x=learn_till, c='r', linestyle='--')